In [1]:
from sampler import ImbalancedDatasetSampler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os

In [2]:
trainImgFolder='data/bihar/bihar_2010_landsat7_cutFiles_rgb_BF_train/'
testImgFolder='data/bihar/bihar_2010_landsat7_cutFiles_rgb_BF_test/'
checkPtFolder=trainImgFolder[:-1]+'_checkpoints'
os.makedirs(checkPtFolder, exist_ok=True)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 2
num_classes = 3
batch_size = 100
learning_rate = 0.001

myTransform = transforms.Compose(
                   [transforms.Resize((64,64)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = torchvision.datasets.ImageFolder(root=trainImgFolder,
                                                transform=myTransform)

test_dataset = torchvision.datasets.ImageFolder(root=testImgFolder,
                                               transform=myTransform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           sampler=ImbalancedDatasetSampler(train_dataset))

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [3]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=3):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(16*16*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    file_name=checkPtFolder+'/'+'epoch_'+str(epoch)+'.ckpt'
    torch.save(model.state_dict(), file_name)

Epoch [1/2], Step [100/315], Loss: 1.0524
Epoch [1/2], Step [200/315], Loss: 0.9196
Epoch [1/2], Step [300/315], Loss: 0.6099
Epoch [2/2], Step [100/315], Loss: 0.5476
Epoch [2/2], Step [200/315], Loss: 0.4739
Epoch [2/2], Step [300/315], Loss: 0.4545


In [5]:
for tk in range(num_epochs):
    print('='*40)
    file_name=checkPtFolder+'/'+'epoch_'+str(tk)+'.ckpt'
    state_dict = torch.load(file_name)
    model.load_state_dict(state_dict)
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        trueLabels=[]
        predictedLabels=[]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            trueLabels.extend(labels.tolist())
            predictedLabels.extend(predicted.tolist())
        print(Counter(predictedLabels))
        print(Counter(trueLabels))
        predictedLabels = np.array(predictedLabels)
        trueLabels = np.array(trueLabels)
        for currentClass in range(3):
            print('='*20)
            maskClass=(predictedLabels==currentClass)
            print('class',currentClass,' accuracy_score: ',accuracy_score(predictedLabels[maskClass],trueLabels[maskClass]))
            print('='*20)
        print('f1_weighted',f1_score(trueLabels, predictedLabels, average='weighted'))
        print('f1_macro',f1_score(trueLabels, predictedLabels, average='macro'))
        print('f1_micro',f1_score(trueLabels, predictedLabels, average='micro'))

Counter({0: 5872, 2: 1585, 1: 407})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9582765667574932
class 1  accuracy_score:  0.0171990171990172
class 2  accuracy_score:  0.07634069400630915
f1_weighted 0.8063688516642774
f1_macro 0.33319370155516204
f1_micro 0.7318158697863683
Counter({0: 4051, 2: 3656, 1: 157})
Counter({0: 7453, 2: 364, 1: 47})
class 0  accuracy_score:  0.9568007899284128
class 1  accuracy_score:  0.012738853503184714
class 2  accuracy_score:  0.05388402625820569
f1_weighted 0.6432884559030715
f1_macro 0.2638234554680306
f1_micro 0.5181841302136317
